In [1]:
import numpy as np
import pytorch_lightning as pl
import xarray as xr
#from torch.utils.data import Dataset, ConcatDataset, DataLoader
from netCDF4 import Dataset
import scipy

import torch
import datetime
from sklearn.feature_extraction import image

data_size          = 1000
N=data_size
batch_size  = 64

DT = 10
DX = 30

Obs_noise = 0.001
Model_noise = 0.02

Database = torch.zeros(N,DX,DT)

#######################################################################################################################
#Définition des fonctions de simulation dynamique :

def f_sum(x) :
    I = torch.rand(N,DX)
    x[:,:,0] = I
    for i in range(N) :
        for k in range(DX):
            for j in range(1,DT):
                if j==1:
                    x[i,k,j] = 0.25*x[i,k,j-1]+0.25*x[i,(k-1)%DX,j-1]+0.25*x[i,(k+1)%DX,j-1]+0.25*x[i,(k+2)%DX,j-1]+0.25*x[i,(k-2)%DX,j-1] +Model_noise*np.random.randn()
                else : 
                    x[i,k,j] = 0.25*x[i,k,j-1]+0.25*x[i,(k-1)%DX,j-1]+0.25*x[i,(k+1)%DX,j-1]+0.25*x[i,(k+2)%DX,j-2]+0.25*x[i,(k-2)%DX,j-2] +Model_noise*np.random.randn()
    return(x)
                
# modèle de somme avec erreur multiplicative
def f_sum2(x) :
    I = torch.rand(N,DX)
    x[:,:,0] = I
    for i in range(N) :
        for k in range(DX):
            for j in range(1,DT):
                if j==1:
                    x[i,k,j] = 0.6*x[i,k,j-1]+0.4*x[i,(k-1)%DX,j-1]+0.2*x[i,(k+1)%DX,j-1]+0.1*x[i,(k+2)%DX,j-1]+0.05*x[i,(k-2)%DX,j-1]+ (x[i,k,j-1]+x[i,(k-1)%DX,j-1])*Model_noise*np.random.randn()
                else : 
                    x[i,k,j] = 0.6*x[i,k,j-1]+0.4*x[i,(k-1)%DX,j-1]+0.2*x[i,(k+1)%DX,j-1]+0.1*x[i,(k+2)%DX,j-2]+0.05*x[i,(k-2)%DX,j-2] +(x[i,k,j-1]+x[i,(k-1)%DX,j-1])*Model_noise*np.random.randn()
    return(x)
    
    
Database = f_sum2(Database)  
print(torch.mean(Database,0))

dataTrainingNoNaND = Database[:int(0.6*N),:,:]
dataValNoNaND = Database[int(0.6*N):int(0.8*N),:,:]
dataTestNoNaND = Database[int(0.8*N):,:,:]


# flagTypeMissData = 0 : reconstruction
# flagTypeMissData = 1 : prévision

flagTypeMissData = 1


if flagTypeMissData ==0 :    
    rateMissingData = 0.5#0.9
           
# create missing data
#flagTypeMissData = 0 : Missing data randomly chosen on the patch driven by rateMissingData
#flagTpeMissData = 1  : Prevision

if flagTypeMissData == 0:
    indRandD         = np.random.permutation(dataTrainingNoNaND.shape[0]*dataTrainingNoNaND.shape[1]*dataTrainingNoNaND.shape[2])
    indRandD         = indRandD[0:int(rateMissingData*len(indRandD))]
    dataTrainingD    = torch.tensor(dataTrainingNoNaND).reshape((dataTrainingNoNaND.shape[0]*dataTrainingNoNaND.shape[1]*dataTrainingNoNaND.shape[2],1))
    dataTrainingD[indRandD] = float('nan')
    dataTrainingD    = torch.reshape(dataTrainingD,(dataTrainingNoNaND.shape[0],dataTrainingNoNaND.shape[1],dataTrainingNoNaND.shape[2]))
            
    indRandD         = np.random.permutation(dataValNoNaND.shape[0]*dataValNoNaND.shape[1]*dataValNoNaND.shape[2])
    indRandD         = indRandD[0:int(rateMissingData*len(indRandD))]
    dataValD    = torch.tensor(dataValNoNaND).reshape((dataValNoNaND.shape[0]*dataValNoNaND.shape[1]*dataValNoNaND.shape[2],1))
    dataValD[indRandD] = float('nan')
    dataValD    = np.reshape(dataValD,(dataValNoNaND.shape[0],dataValNoNaND.shape[1],dataValNoNaND.shape[2]))
            
            
    indRandD         = np.random.permutation(dataTestNoNaND.shape[0]*dataTestNoNaND.shape[1]*dataTestNoNaND.shape[2])
    indRandD         = indRandD[0:int(rateMissingData*len(indRandD))]
    dataTestD        = torch.tensor(dataTestNoNaND).reshape((dataTestNoNaND.shape[0]*dataTestNoNaND.shape[1]*dataTestNoNaND.shape[2],1))
    dataTestD[indRandD] = float('nan')
    dataTestD          = np.reshape(dataTestD,(dataTestNoNaND.shape[0],dataTestNoNaND.shape[1],dataTestNoNaND.shape[2]))

    genSuffixObs    = '_ObsRnd_%02d_%02d'%(100*rateMissingData,10*Model_noise**2)
 
else :
    frac_prev = 2/3  #Proportion de jours dont on a les observations
    
    dataTrainingD    = torch.zeros((dataTrainingNoNaND.shape))
    dataTrainingD[:] = float('nan')
            
    dataValD    = torch.zeros((dataValNoNaND.shape))
    dataValD[:] = float('nan')
            
    dataTestD        = torch.zeros((dataTestNoNaND.shape))
    dataTestD[:]     = float('nan')
    
    dataTrainingD[:,:,:(int(2*DT/3+1))] = dataTrainingNoNaND[:,:,:(int(frac_prev*DT+1))]
    dataValD[:,:,:(int(2*DT/3+1))] = dataValNoNaND[:,:,:(int(frac_prev*DT+1))]
    dataTestD[:,:,:(int(2*DT/3+1))] = dataTestNoNaND[:,:,:(int(frac_prev*DT+1))]
    genSuffixObs    = '_ObsSubRnd_%02d_%02d'%(100*Obs_noise,10*Model_noise**2)
    
print('... Data type: '+genSuffixObs)
    #for nn in range(0,dataTraining.shape[1],time_step_obs):
    #    dataTraining[:,::time_step_obs,:] = dataTrainingNoNaN[:,::time_step_obs,:]
    #dataTest    = np.zeros((dataTestNoNaN.shape))
    #dataTest[:] = float('nan')
    #dataTest[:,::time_step_obs,:] = dataTestNoNaN[:,::time_step_obs,:]

            
                                
# mask for NaN
maskTrainingD = (dataTrainingD == dataTrainingD).type_as(dataTrainingD)
maskValD = (dataValD == dataValD).type_as(dataTrainingD)
maskTestD     = ( dataTestD    ==  dataTestD   ).type_as(dataTrainingD)
            
dataTrainingD = torch.nan_to_num(dataTrainingD)       
dataValD = torch.nan_to_num(dataValD)
dataTestD     = torch.nan_to_num(dataTestD)
            
    # Permutation to have channel as #1 component
dataTrainingD      = torch.moveaxis(dataTrainingD,-1,1)
maskTrainingD      = torch.moveaxis(maskTrainingD,-1,1)
dataTrainingNoNaND = torch.moveaxis(dataTrainingNoNaND,-1,1)
        
dataValD      = torch.moveaxis(dataValD,-1,1)
maskValD      = torch.moveaxis(maskValD,-1,1)
dataValNoNaND = torch.moveaxis(dataValNoNaND,-1,1)
            
dataTestD      = torch.moveaxis(dataTestD,-1,1)
maskTestD      = torch.moveaxis(maskTestD,-1,1)
dataTestNoNaND = torch.moveaxis(dataTestNoNaND,-1,1)
            
# set to NaN patch boundaries
#dataTraining[:,0:5,:] =  dataTrainingNoNaN[:,0:5,:]
#dataTest[:,0:5,:]     =  dataTestNoNaN[:,0:5,:]
    
############################################
## raw data
X_trainD         = dataTrainingNoNaND       
X_train_missingD = dataTrainingD
mask_trainD      = maskTrainingD
        
X_valD         = dataValNoNaND
X_val_missingD = dataValD
mask_valD      = maskValD
        
X_testD         = dataTestNoNaND
X_test_missingD = dataTestD
mask_testD      = maskTestD


############################################
## normalized data wrt to each measurement station

if flagTypeMissData == 0:    
    mean_Tr= torch.mean(X_train_missingD[:],0)/torch.mean(mask_trainD,0)
    std_Tr =torch.std((X_train_missingD-mean_Tr)*mask_trainD,0)/torch.sqrt((torch.mean(mask_trainD,0)))

    x_train_missingD = (X_train_missingD - mean_Tr*mask_trainD)*(1/std_Tr*mask_trainD)           
    x_val_missingD = (X_val_missingD - mean_Tr*mask_valD)*(1/std_Tr*mask_valD)
    x_test_missingD  = (X_test_missingD - mean_Tr*mask_testD)*(1/std_Tr*mask_testD)

    x_trainD = (X_trainD - mean_Tr) / std_Tr
    x_valD = (X_valD - mean_Tr) / std_Tr
    x_testD  = (X_testD - mean_Tr) / std_Tr

    # Generate noisy observsation
        
    X_train_obsD = X_train_missingD + Obs_noise * maskTrainingD * torch.randn(X_train_missingD.shape[0],X_train_missingD.shape[1],X_train_missingD.shape[2])
    X_val_obsD = X_val_missingD + Obs_noise * maskValD * torch.randn(X_val_missingD.shape[0],X_val_missingD.shape[1],X_val_missingD.shape[2])
    X_test_obsD  = X_test_missingD  + Obs_noise * maskTestD * torch.randn(X_test_missingD.shape[0],X_test_missingD.shape[1],X_test_missingD.shape[2])

    mean_Tr_obs= torch.mean(X_train_obsD[:],0)/torch.mean(mask_trainD,0)
    std_Tr_obs = torch.std((X_train_obsD-mean_Tr_obs)*mask_trainD,0)/torch.sqrt((torch.mean(mask_trainD,0)))
            
    x_train_obsD = (X_train_obsD - mean_Tr_obs*mask_trainD)*(1/std_Tr_obs*mask_trainD)
    x_val_obsD = (X_val_obsD - mean_Tr_obs*mask_valD)*(1/std_Tr_obs*mask_valD)
    x_test_obsD  = (X_test_obsD - mean_Tr_obs*mask_testD)*(1/std_Tr_obs*mask_testD)

    
elif flagTypeMissData==1 : 
    mean_Tr= torch.mean(X_trainD[:],0)
    std_Tr =torch.std((X_trainD-mean_Tr),0)

    x_train_missingD = (X_train_missingD - mean_Tr*mask_trainD)/std_Tr
    x_val_missingD = (X_val_missingD - mean_Tr*mask_valD)/std_Tr
    x_test_missingD  = (X_test_missingD - mean_Tr*mask_testD)/std_Tr

    x_trainD = (X_trainD - mean_Tr) /std_Tr
    x_valD = (X_valD - mean_Tr) / std_Tr
    x_testD  = (X_testD - mean_Tr) / std_Tr

    # Generate noisy observsation
        
    X_train_obsD = X_train_missingD + Obs_noise * maskTrainingD * torch.randn(X_train_missingD.shape[0],X_train_missingD.shape[1],X_train_missingD.shape[2])
    X_val_obsD = X_val_missingD + Obs_noise * maskValD * torch.randn(X_val_missingD.shape[0],X_val_missingD.shape[1],X_val_missingD.shape[2])
    X_test_obsD  = X_test_missingD  + Obs_noise * maskTestD * torch.randn(X_test_missingD.shape[0],X_test_missingD.shape[1],X_test_missingD.shape[2])

    mean_Tr_obs= torch.mean(X_train_obsD[:],0)
    std_Tr_obs = torch.nan_to_num(torch.std((X_train_obsD-mean_Tr_obs),0)/torch.sqrt((torch.mean(mask_trainD,0))),1)
            
    x_train_obsD = (X_train_obsD - mean_Tr_obs)/std_Tr_obs
    x_val_obsD = (X_val_obsD - mean_Tr_obs)/std_Tr_obs
    x_test_obsD  = (X_test_obsD - mean_Tr_obs*mask_testD)/std_Tr_obs

          
print('..... Training dataset: %dx%dx%d'%(x_train_missingD.shape[0],x_trainD.shape[1],x_trainD.shape[2]))
print('..... Validation dataset: %dx%dx%d'%(x_valD.shape[0],x_valD.shape[1],x_valD.shape[2]))
print('..... Test dataset    : %dx%dx%d'%(x_testD.shape[0],x_testD.shape[1],x_testD.shape[2]))
            

print('........ Initialize interpolated states')
## Initial interpolation
#flagInit = 0 : Input values for the forced normal distribution
#flagInit = 1 : Input values for the forced GPD distribution


tensor([[ 0.5179,  1.0575,  0.9942,  0.7949,  0.6359,  0.5087,  0.4073,  0.3258,
          0.2603,  0.2082],
        [ 0.4935,  1.0487,  1.6199,  1.8914,  1.9899,  1.9715,  1.8822,  1.7500,
          1.5952,  1.4307],
        [ 0.4861,  1.0309,  1.6042,  2.3596,  3.1269,  3.7729,  4.2710,  4.5946,
          4.7685,  4.8106],
        [ 0.4942,  1.0336,  1.6001,  2.3461,  3.4591,  4.8345,  6.3362,  7.8328,
          9.2139, 10.3934],
        [ 0.4861,  1.0383,  1.5982,  2.3437,  3.4509,  5.0699,  7.2704,  9.9995,
         13.1287, 16.4959],
        [ 0.5159,  1.0526,  1.6184,  2.3572,  3.4502,  5.0610,  7.4374, 10.7857,
         15.2558, 20.8771],
        [ 0.4898,  1.0579,  1.6277,  2.3747,  3.4686,  5.0794,  7.4459, 10.9305,
         15.9607, 22.9687],
        [ 0.5142,  1.0609,  1.6365,  2.3906,  3.4999,  5.1150,  7.4776, 10.9644,
         16.0893, 23.5306],
        [ 0.5118,  1.0663,  1.6400,  2.4010,  3.5220,  5.1609,  7.5360, 11.0196,
         16.1358, 23.6637],
        [ 0.4986,  

In [2]:
print(x_train_obsD[0,:,:])

tensor([[ 0.1307, -0.8808,  0.5696, -1.1310, -0.5547,  1.4900,  0.5842,  0.0422,
         -0.3590, -1.5406,  0.7695,  0.8374,  0.5145,  1.2259, -1.2064, -1.4560,
          0.0355,  1.6972,  0.5421, -1.5470, -1.1869, -0.9447, -0.2472,  0.9672,
         -0.2495,  0.2906, -1.6426, -0.4022,  0.9639,  0.0946],
        [-0.0141, -0.5400, -0.5267, -0.4997, -0.3064,  0.9176,  1.1888,  0.0506,
         -0.5369, -0.8508,  0.1045,  1.2444,  1.1722,  0.4837, -0.7922, -1.3082,
         -0.1923,  0.9510,  0.7299, -1.2651, -2.0562, -1.4542, -0.5106,  0.3494,
          0.1406, -0.5468, -1.0539, -0.8008,  0.2919,  0.5568],
        [ 0.2406, -0.4072, -0.7088, -0.2951, -0.3298,  0.4645,  0.9692,  0.1728,
         -0.1267, -0.7505, -0.1369,  0.8092,  1.1616,  0.5974, -0.2953, -0.9060,
         -0.7306,  0.4720,  0.7099, -0.5465, -1.6629, -1.7577, -1.0143, -0.0404,
         -0.1118, -0.3316, -0.7066, -0.8361, -0.2568,  0.0497],
        [ 0.2952, -0.2718, -0.4828, -0.5344, -0.3313,  0.1324,  0.8560,  0.6587

In [3]:

flagInit = 0
   
if flagInit == 0: 
    
    
    
    if flagTypeMissData == 0:
        X_train_InitD = torch.zeros(int(0.6*N),DT,DX,2)
        X_train_InitD[:,:,:,1] =torch.ones(int(0.6*N),DT,DX)
        X_val_InitD = torch.zeros(int(0.2*N),DT,DX,2)
        X_val_InitD[:,:,:,1] =torch.ones(int(0.2*N),DT,DX)
        X_test_InitD = torch.zeros(int(0.2*N),DT,DX,2)
        X_test_InitD[:,:,:,1] =torch.ones(int(0.2*N),DT,DX)
        
        for ii in range(0,X_trainD.shape[0]):
        # Initial linear interpolation for each component
            XInitD = torch.zeros((X_trainD.shape[1],X_trainD.shape[2]))
           
            for kk in range(0,mask_trainD.shape[1]):
                indt  = torch.where( mask_trainD[ii,kk,:] == 1.0 )[0]
                indt_ = torch.where( mask_trainD[ii,kk,:] == 0.0 )[0]
           
                if len(indt) > 1:
                    indt_[ torch.where( indt_ <torch.min(indt)) ] = torch.min(indt)
                    indt_[ torch.where( indt_ > torch.max(indt)) ] = torch.max(indt)
                    fkk = scipy.interpolate.interp1d(indt, X_train_obsD[ii,kk,indt])
                    XInitD[kk,indt]  = X_train_obsD[ii,kk,indt]
                    XInitD[kk,indt_] = torch.Tensor(fkk(indt_))
                else:
                    XInitD = XInitD + meanTr
            
            X_train_InitD[ii,:,:,0] = XInitD
            
        for ii in range(0,X_valD.shape[0]):
            # Initial linear interpolation for each component
            XInitD = torch.zeros((X_valD.shape[1],X_valD.shape[2]))
           
            for kk in range(0,mask_valD.shape[1]):
                indt  = torch.where( mask_valD[ii,kk,:] == 1.0 )[0]
                indt_ = torch.where( mask_valD[ii,kk,:] == 0.0 )[0]
           
                if len(indt) > 1:
                    indt_[ torch.where( indt_ < torch.min(indt)) ] = torch.min(indt)
                    indt_[ torch.where( indt_ > torch.max(indt)) ] = torch.max(indt)
                    fkk = scipy.interpolate.interp1d(indt, X_val_obsD[ii,kk,indt])
                    XInitD[kk,indt]  = X_val_obsD[ii,kk,indt]
                    XInitD[kk,indt_] = torch.Tensor(fkk(indt_))
                else:
                    XInitD = XInitD + meanTr
            
            X_val_InitD[ii,:,:,0] = XInitD
            

        for ii in range(0,X_testD.shape[0]):
        # Initial linear interpolation for each component
            XInit = torch.zeros((X_testD.shape[1],X_testD.shape[2]))
            
            for kk in range(0,X_testD.shape[1]):
                indt  = torch.where( mask_testD[ii,kk,:] == 1.0 )[0]
                indt_ = torch.where( mask_testD[ii,kk,:] == 0.0 )[0]
            
                if len(indt) > 1:
                    indt_[ torch.where( indt_ < torch.min(indt)) ] = torch.min(indt)
                    indt_[ torch.where( indt_ > torch.max(indt)) ] = torch.max(indt)
                    fkk = scipy.interpolate.interp1d(indt, X_test_obsD[ii,kk,indt])
                    XInit[kk,indt]  = X_test_obsD[ii,kk,indt]
                    XInit[kk,indt_] = torch.Tensor(fkk(indt_))
                else:
                    XInit = XInit + meanTr
        
            X_test_InitD[ii,:,:,0] = XInit
        
    elif flagTypeMissData == 1:
        X_train_InitD = torch.zeros(int(0.6*N),DT,DX,2)
        X_train_InitD[:,:,:,1] =torch.ones(int(0.6*N),DT,DX)
        X_val_InitD = torch.zeros(int(0.2*N),DT,DX,2)
        X_val_InitD[:,:,:,1] =torch.ones(int(0.2*N),DT,DX)
        X_test_InitD = torch.zeros(int(0.2*N),DT,DX,2)
        X_test_InitD[:,:,:,1] =torch.ones(int(0.2*N),DT,DX)
        X_train_InitD[:,:,:,0] = X_train_obsD
        XInitD = X_train_InitD[:,int(frac_prev*DT),:,0]
        for i in range(int(frac_prev*DT)+1,DT):
            X_train_InitD[:,i,:,0] = XInitD
            
        X_val_InitD[:,:,:,0] = X_val_obsD
        XInitD = X_val_InitD[:,int(frac_prev*DT),:,0]
        for i in range(int(frac_prev*DT)+1,DT):
            X_val_InitD[:,i,:,0] = XInitD
        
        X_test_InitD[:,:,:,0] = X_test_obsD
        XInitD = X_test_InitD[:,int(frac_prev*DT),:,0]
        for i in range(int(frac_prev*DT)+1,DT):
            X_test_InitD[:,i,:,0] = XInitD
         

In [6]:
import matplotlib.pyplot as plt

print(X_train_InitD.shape)
print(X_train_obsD[0,:,:])
print(X_train_InitD[0,:,0,0])

torch.Size([600, 10, 30, 2])
tensor([[ 0.4203,  0.3611,  0.2132,  0.9081,  0.5133,  0.2905,  0.1740,  0.3424,
          0.7795,  0.8470,  0.4079,  0.6792,  0.1670,  0.3566,  0.1522,  0.5931,
          0.4796,  0.0619,  0.3287,  0.2269,  0.2807,  0.9535,  0.5963,  0.8767,
          0.3873,  0.3986,  0.1652,  0.2574,  0.5176,  0.3703],
        [ 0.4749,  0.5599,  0.6195,  0.5969,  0.5094,  0.5315,  0.5246,  0.6118,
          0.6353,  0.7751,  0.7374,  0.5927,  0.4399,  0.4516,  0.4030,  0.4013,
          0.3857,  0.4387,  0.3340,  0.4377,  0.6098,  0.7276,  0.7728,  0.8367,
          0.6059,  0.5271,  0.4342,  0.4189,  0.3643,  0.4931],
        [ 0.2888,  0.6050,  0.5136,  0.4810,  0.3660,  0.5760,  0.6235,  0.5786,
          0.4838,  0.6070,  0.6295,  0.6313,  0.3925,  0.5593,  0.3761,  0.3340,
          0.3019,  0.4061,  0.3646,  0.4425,  0.4947,  0.6054,  0.5383,  0.7492,
          0.5779,  0.5577,  0.4901,  0.4125,  0.3552,  0.5071],
        [ 0.0658,  0.2283,  0.4018,  0.3621,  0.32

In [7]:
         
x_train_InitD = torch.zeros(X_train_InitD.shape[0],DT,DX,2)
x_val_InitD = torch.zeros(X_val_InitD.shape[0],DT,DX,2)
x_test_InitD = torch.zeros(X_test_InitD.shape[0],DT,DX,2) 

x_train_InitD[:,:,:,0] = ( X_train_InitD[:,:,:,0] - mean_Tr ) / std_Tr
x_val_InitD[:,:,:,0] = ( X_val_InitD[:,:,:,0] - mean_Tr ) / std_Tr
x_test_InitD[:,:,:,0] = ( X_test_InitD[:,:,:,0] - mean_Tr ) / std_Tr
x_train_InitD[:,:,:,1] = X_train_InitD[:,:,:,1]                
x_val_InitD[:,:,:,1] = X_val_InitD[:,:,:,1] 
x_test_InitD[:,:,:,1] = X_test_InitD[:,:,:,1] 

print('..... Training dataset: %dx%dx%d'%(x_trainD.shape[0],x_trainD.shape[1],x_trainD.shape[2]))
print('..... Validation dataset: %dx%dx%d'%(x_valD.shape[0],x_valD.shape[1],x_valD.shape[2]))
print('..... Test dataset    : %dx%dx%d'%(x_testD.shape[0],x_testD.shape[1],x_testD.shape[2]))
    
    
training_dataset     = torch.utils.data.TensorDataset(torch.Tensor(X_train_InitD),torch.Tensor(X_train_obsD),torch.Tensor(mask_trainD),torch.Tensor(X_trainD)) # create your datset

val_dataset         = torch.utils.data.TensorDataset(torch.Tensor(X_val_InitD),torch.Tensor(X_val_obsD),torch.Tensor(mask_valD),torch.Tensor(X_valD)) # create your datset
     
test_dataset         = torch.utils.data.TensorDataset(torch.Tensor(X_test_InitD),torch.Tensor(X_test_obsD),torch.Tensor(mask_testD),torch.Tensor(X_testD)) # create your datset

        
dataloaders = {
                'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True),
                'val': torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True),
                'test': torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True),
            }            
dataset_sizes = {'train': len(training_dataset),'val': len(val_dataset), 'test': len(test_dataset)}

..... Training dataset: 600x10x30
..... Validation dataset: 200x10x30
..... Test dataset    : 200x10x30


In [12]:
print(mean_Tr)
print(std_Tr)

tensor([[ 5.1287e-01,  4.9230e-01,  4.9898e-01,  4.9515e-01,  5.0145e-01,
          4.9898e-01,  4.6803e-01,  4.8220e-01,  5.0688e-01,  5.0020e-01,
          5.0905e-01,  4.9385e-01,  5.1194e-01,  5.2483e-01,  4.9832e-01,
          5.0417e-01,  5.2948e-01,  5.0992e-01,  4.9538e-01,  4.9474e-01,
          4.9964e-01,  4.9871e-01,  5.0497e-01,  5.0564e-01,  4.9177e-01,
          5.0638e-01,  4.8383e-01,  4.8813e-01,  5.1658e-01,  5.0337e-01],
        [ 6.3137e-01,  6.2522e-01,  6.2608e-01,  6.2081e-01,  6.1596e-01,
          6.0951e-01,  6.1294e-01,  6.1426e-01,  6.1523e-01,  6.2217e-01,
          6.3038e-01,  6.3407e-01,  6.3610e-01,  6.3455e-01,  6.4451e-01,
          6.4217e-01,  6.3394e-01,  6.3385e-01,  6.3345e-01,  6.2394e-01,
          6.2209e-01,  6.2587e-01,  6.2558e-01,  6.2655e-01,  6.2273e-01,
          6.1903e-01,  6.2189e-01,  6.2500e-01,  6.2642e-01,  6.2854e-01],
        [ 4.1206e-01,  5.6436e-01,  5.6685e-01,  5.5989e-01,  5.5013e-01,
          5.4837e-01,  5.5812e-01,  